<a href="https://colab.research.google.com/github/JonathanACastro/PySpark/blob/main/Lag_lead_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Installing PySpark in Google Colab

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
# import pyspark.sql.types as T
# import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [92.1 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3,422 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2,322 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/restricted amd64 Packages [1,202 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security 

In [13]:
from pyspark.sql.functions import *
from pyspark.sql.functions import date_add
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql.dataframe import DataFrame
from typing import List
import datetime
import time

In [3]:
spark = SparkSession.builder.getOrCreate()

In [48]:
def _to_date_time(value):
 return datetime.datetime.strptime(value, '%Y-%m-%d %H:%M') if(value) else None

schema = ['store', 'code_store', 'description','price','validFrom', 'validTo']
data = [('target', 1000, 'coffe', 2, _to_date_time('2022-09-10 00:00'), _to_date_time('2022-09-16 00:00')),
        ('target', 1000, 'coffe', 3, _to_date_time('2022-09-16 00:00'), _to_date_time('2022-09-25 00:00')),
        ('target', 1000, 'coffe', 5, _to_date_time('2022-09-25 00:00'), None)]
df_input = spark.createDataFrame(data, schema)

In [49]:
windowSpec = Window.partitionBy("store", "code_store", "description").orderBy(asc("validFrom"))

df_input = (df_input.withColumn("lead_validFrom", lead("validFrom", 1).over(windowSpec))#compare with next row
        .withColumn("lag_validFrom", lag("validFrom", 1).over(windowSpec))#compare with previews row 
      )

In [50]:
df_input.show()

+------+----------+-----------+-----+-------------------+-------------------+-------------------+-------------------+
| store|code_store|description|price|          validFrom|            validTo|     lead_validFrom|      lag_validFrom|
+------+----------+-----------+-----+-------------------+-------------------+-------------------+-------------------+
|target|      1000|      coffe|    2|2022-09-10 00:00:00|2022-09-16 00:00:00|2022-09-16 00:00:00|               null|
|target|      1000|      coffe|    3|2022-09-16 00:00:00|2022-09-25 00:00:00|2022-09-25 00:00:00|2022-09-10 00:00:00|
|target|      1000|      coffe|    5|2022-09-25 00:00:00|               null|               null|2022-09-16 00:00:00|
+------+----------+-----------+-----+-------------------+-------------------+-------------------+-------------------+



In [53]:
#To roll back one day of validTo we need to compare two dates, validTo == lead_validFrom
df_input = (df_input
      .withColumn("CheckIsEqual_from_to", col("validTo")==col("lead_validFrom"))
      .withColumn("validTo", when(col("CheckIsEqual_from_to") == "true", date_add(df_input.validTo,-1).cast(TimestampType())))#here we back one day -1
      )
#we drop the columns that don't will be used anymore
df_output = df_input.drop(*["lead_validfrom", "lag_validFrom", "CheckIsEqual_from_to"]) 

In [54]:
df_output.show()

+------+----------+-----------+-----+-------------------+-------------------+
| store|code_store|description|price|          validFrom|            validTo|
+------+----------+-----------+-----+-------------------+-------------------+
|target|      1000|      coffe|    2|2022-09-10 00:00:00|2022-09-15 00:00:00|
|target|      1000|      coffe|    3|2022-09-16 00:00:00|2022-09-24 00:00:00|
|target|      1000|      coffe|    5|2022-09-25 00:00:00|               null|
+------+----------+-----------+-----+-------------------+-------------------+

